This notebook notebook runs a full grid evaluation of the E-field distributional landscape given input parameters $(x,y,\theta)$. This is achieved via massive experiment parallelization using sbatch then collection into a local directory via file deposition.  

In [1]:
#Load custom slurm magic
%load_ext slurm_magic

### Properly Batched Version using Internal Parallelization

In [1]:
30**3

27000

In [2]:
%%sbatch 
#!/usr/bin/env python
#SBATCH --partition=high-moby
#SBATCH --output=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.out
#SBATCH --error=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.err
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=5G
#SBATCH --job-name=field_simulations
#SBATCH --array=0
#SBATCH --time=0:30:00

import sys
import os
import numpy as np
from fieldopt import geolib
from fieldopt.objective import FieldFunc

#Directory specification
proj_dir =  '/projects/jjeyachandra/rtms_optimize'
mesh_file = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
coil_file = os.path.join(proj_dir,'resources','coils','Magstim_70mm_Fig8.nii.gz')
tet_file = os.path.join(proj_dir,'output','tetra_parcels')
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
output_dir = os.path.join(proj_dir,'objective','scores')

C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

#Load in element mask
p_map = np.load(os.path.join(proj_dir,'output','tetra_parcels.npy'))[:,2]

f = FieldFunc(mesh_file=mesh_file, quad_surf_consts=C,
              surf_to_mesh_matrix=iR, tet_weights=p_map,
              field_dir='/tmp/', coil=coil_file, cpus=6)

#Set up evaluation grid
fidelity = 30
x_samps = np.linspace(b[0,0],b[0,1],fidelity)
y_samps = np.linspace(b[1,0],b[1,1],fidelity)
rot_samps = np.linspace(0,180,fidelity)
XX,YY,RR = np.meshgrid(x_samps,y_samps,rot_samps)
XX = XX.flatten().reshape((fidelity**3,1))
YY = YY.flatten().reshape((fidelity**3,1))
RR = RR.flatten().reshape((fidelity**3,1))
sample_set = np.concatenate((XX,YY,RR),axis=1)
ind = int(os.environ['SLURM_ARRAY_TASK_ID'])

#Evaluate to extract score
score = f.evaluate([sample_set[ind]])
score_dir=os.path.join(proj_dir,'objective_grid','scores')
score_file=os.path.join(score_dir,'fx_{}'.format(ind))
np.save(score_file,score)

print('Successfully finished simulation with following parameters:')
print('INPUTS: {} {} {}'.format(sample_set[0][0],sample_set[0][1],sample_set[0][2]))
print('INDEX: {}'.format(ind))
print('SCORE:',score)
print('Finished successfully')

u'Submitted batch job 81687\n'

### Quick Parallelization
This method of parallelization assigns a batch job per simulation

In [ ]:
%%sbatch 
#!/usr/bin/env python
#SBATCH --partition=high-moby
#SBATCH --output=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.out
#SBATCH --error=/projects/jjeyachandra/rtms_optimize/objective_grid/log/sim_%a.err
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=5G
#SBATCH --job-name=field_simulations
#SBATCH --array=3000-3200
#SBATCH --time=0:30:00

#Package import
import sys
sys.path.insert(0,'/projects/jjeyachandra/simnibs/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/projects/jjeyachandra/simnibs/Python_modules/src')

import os
from shutil import copy

import numpy as np
from simnibs import sim_struct, run_simulation
from fieldopt import geolib, tetrapro

####### DATA LOADING

#File specification

proj_dir = '/projects/jjeyachandra/rtms_optimize'
#local_dir = '/home/jjeyachandra/'
local_dir=os.path.join(proj_dir,'data','simnibs_output')
C = np.fromfile(os.path.join(proj_dir,'output','quadratic_vec'))
iR = np.fromfile(os.path.join(proj_dir,'output','inverse_rot')).reshape(3,3)
b = np.fromfile(os.path.join(proj_dir,'output','param_bounds')).reshape(3,2)
grid_path = os.path.join(proj_dir,'objective_grid','grid')

###### INPUT SETUP 

#Set up evaluation grid
fidelity = 20
x_samps = np.linspace(b[0,0],b[0,1],fidelity)
y_samps = np.linspace(b[1,0],b[1,1],fidelity)
rot_samps = np.linspace(0,180,fidelity)
XX,YY,RR = np.meshgrid(x_samps,y_samps,rot_samps)
XX = XX.flatten().reshape((fidelity**3,1))
YY = YY.flatten().reshape((fidelity**3,1))
RR = RR.flatten().reshape((fidelity**3,1))
sample_set = np.concatenate((XX,YY,RR),axis=1)

#Pick input parameters
ind = int(os.environ['SLURM_ARRAY_TASK_ID'])
x,y,r = sample_set[ind,:] 

#Set up spatial postioning
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Set up rotational positioning
preaff_rot, preaff_n = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)
n = np.matmul(iR,preaff_n)

###### SIMULATION SETUP

#Start a session
S = sim_struct.SESSION()
S.fnamehead = os.path.join(local_dir,'sub-CMH090.msh')
S.pathfem = os.path.join(grid_path,'sim_{}'.format(ind))
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

print('Running simulation with:')
print('INPUTS: {} {} {}'.format(x,y,r))
print('INDEX: {}'.format(ind))
print('POSITION ARRAY:')
print(pos.matsimnibs)

#Run simulation
try:
    run_simulation(S)
except IOError:
    print('Found existing simnibs run, skipping simulation!')
    pass

###### PROCESS OUTPUTS 

#Save the geo file
geo_dir=os.path.join(proj_dir,'objective_grid','geo_output')
geo_file=os.path.join(grid_path,'sim_{}'.format(ind),
                      'sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_coil_pos.geo')
geo_out=os.path.join(geo_dir,'sub-CMH090_{}.geo'.format(ind))
copy(geo_file,geo_out)

#Compute score on output
field_file=os.path.join(S.pathfem,'sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh')
parcelmap_path = os.path.join(proj_dir,'output','tetra_parcels')
num_parcels = 20
target_parcel = 2
proj_map = np.fromfile(parcelmap_path)
proj_map = proj_map.reshape( (proj_map.shape[0]//(num_parcels + 1), num_parcels + 1) )
_, elem_ids, _ = geolib.load_gmsh_elems(S.fnamehead,entity=(3,2))
norm_E = geolib.get_field_subset(field_file,elem_ids)
score = geolib.compute_field_score(norm_E,proj_map,target_parcel)

#Save score value
score_dir=os.path.join(proj_dir,'objective_grid','scores')
score_file=os.path.join(score_dir,'fx_{}'.format(ind))
np.save(score_file,score)

#Print score value
print('SCORE:',score)

print('Finished successfully')

In [ ]:
#Block for evaluating scoring functions when simulation results are already available
import numpy as np
from fieldopt import geolib

proj_dir = '/projects/jjeyachandra/rtms_optimize'
local_dir=os.path.join(proj_dir,'data','simnibs_output')
fnamehead = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')

for i in np.arange(3000,3200):
    
    pathfem = os.path.join(proj_dir,'objective_grid','grid','sim_{}'.format(i))
    score_dir=os.path.join(proj_dir,'objective_grid','scores')
    score_file=os.path.join(score_dir,'fx_{}'.format(i))
    
    if os.path.exists(score_file):
        print('Found:',pathfem)
        continue
    
    
    print('Analyzing:',pathfem)
    field_file=os.path.join(pathfem,'sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh')
    parcelmap_path = os.path.join(proj_dir,'output','tetra_parcels')
    num_parcels = 20
    target_parcel = 2
    proj_map = np.fromfile(parcelmap_path)
    proj_map = proj_map.reshape( (proj_map.shape[0]//(num_parcels + 1), num_parcels + 1) )
    _, elem_ids, _ = geolib.load_gmsh_elems(fnamehead,entity=(3,2))
    norm_E = geolib.get_field_subset(field_file,elem_ids)
    score = geolib.compute_field_score(norm_E,proj_map,target_parcel)
    
    np.save(score_file,score)